# Interactive Dashboard for Sales Data Analysis with AI-generated insights and summaries. 
Build with OpenAI Library and DASH

### Process: 
1. Load and Explore Data
- 1.1. Import necessary libraries.
- 1.2. Load the dataset.
- 1.3. Check the shape of the dataset, Duplicate Rows Number, Duplicate Rows Number.
- 1.4. Assess data quality: identify missing values and % of missing values.

2. Preprocess Data
- 2.1. Dealing with Duplicates
- 2.2. Standardize Column Names:
- Remove leading and trailing spaces and underscores.
- Replace non-alphanumeric characters with underscores.
- Convert column names to lowercase.

3. Data Types Correction
- Ensure columns have the correct data types.
- Parse dates: Convert dates to datetime format.

4. Data Correction
- 4.1. Identify data that need correction: values_count(), unique()
- 4.2. Correcting Typos, Handling Synonyms, Standardizing Formats (countries, states, cities etc.)
- 4.3. Export cleaned dataset

5. OpenAI Library
- 5.1. Installation anf of the OpenAI Library
- 5.2. Reading API Configuration Keys

6. Build the Dashboard in DASH

### 1. Load and Explore Data
- 1.1. Import necessary libraries.
- 1.2. Load the dataset.
- 1.3. Check the shape of the dataset, Duplicate Rows Number, Duplicate Rows Number.
- 1.4. Assess data quality: identify missing values and % of missing values.

In [1]:
# 1.1. Import necessary libraries.
import openai
import os
import dash
import plotly.express as px
import re
import dash
from dash import dcc, html
from dotenv import load_dotenv
import pandas as pd
from dash import html, dcc
from dash.dependencies import Input, Output, State

In [2]:
# 1.2. Load the dataset
df = pd.read_csv('../GPT_OPEN_AI_project/data/raw_data/sales_history_sample_june.csv')

In [3]:
# 1.3. Check the shape of the dataset, Duplicate Rows Number, Duplicate Rows Number.
def initial_data_checking(df):
    # Print the shape of the DataFrame (number of rows and columns)
    print("\nShape of the DataFrame:\n")
    print(df.shape)

    # Print the count of duplicate rows
    print("\nDuplicate Rows Number:\n")
    print(df.duplicated().sum())

    # Print summary statistics for numerical columns
    print("\nSummary Statistics:\n")
    print(df.describe())
    
initial_data_checking(df)


Shape of the DataFrame:

(1119, 13)

Duplicate Rows Number:

68

Summary Statistics:

         Invoice No      Order No      Quantity     Unit Cost    Unit Price  \
count  1.119000e+03  1.119000e+03   1119.000000   1119.000000   1119.000000   
mean   2.151564e+07  2.401454e+07    210.370078    109.530577    215.318618   
std    7.454987e+06  3.445960e+05    904.026709   1060.981943   1669.957269   
min    3.169850e+05  2.125118e+07      0.996000      0.000000      0.000000   
25%    2.400206e+07  2.410055e+07      1.000000      0.393150      1.675000   
50%    2.400220e+07  2.410141e+07     24.000000      9.401800     21.690000   
75%    2.440108e+07  2.420047e+07    100.000000     26.632000     79.287000   
max    2.440123e+07  2.425048e+07  16072.000000  28588.005000  43378.832000   

       Total Value USD  
count      1119.000000  
mean       4965.630920  
std       16270.141993  
min           0.000000  
25%         132.500000  
50%         962.000000  
75%        3378.000000  
m

In [4]:
# 1.4. Assess data quality: identify missing values and % of missing values.
def unique_and_missing_values_dtype(df):
    # Non-null counts and data types
    non_null_counts = df.notnull().sum()
    dtypes = df.dtypes

    # Count of unique values
    unique_count = df.nunique()

    # Percentage of unique values
    unique_percentage = (df.nunique() / len(df)) * 100

    # Count of missing values
    missing_count = df.isnull().sum()

    # Percentage of missing values
    missing_percentage = df.isnull().mean() * 100

    # Combine into a DataFrame
    summary = pd.DataFrame({
        'non-Null_count': non_null_counts,
        'dtype': dtypes,
        'unique_values': unique_count,
        '%_unique': unique_percentage.round(2).astype(str) + '%',
        'missing_values': missing_count,
        '%_missing': missing_percentage.round(2).astype(str) + '%'
    })

    return summary

unique_and_missing_values_dtype(df)

,non-Null_count,dtype,unique_values,%_unique,missing_values,%_missing
Invoice No,1119,int64,613,54.78%,0,0.0%
Order No,1119,int64,566,50.58%,0,0.0%
Sales Rep Name,1117,object,15,1.34%,2,0.18%
SITE,1119,object,3,0.27%,0,0.0%
Date,1119,object,20,1.79%,0,0.0%
Ship to Country,1119,object,9,0.8%,0,0.0%
Market Segment,1119,object,22,1.97%,0,0.0%
Quantity,1119,float64,180,16.09%,0,0.0%
Unit Cost,1119,float64,638,57.02%,0,0.0%
Unit Price,1119,float64,601,53.71%,0,0.0%


Cleaning Summary:

1. Sales Rep Name drop missing values rows
2. Format Date to datetime
3. Format Quantity to int

### 2. Preprocess Data
- 2.1. Dealing with Duplicates
- 2.2. Standardize Column Names:
- Remove leading and trailing spaces and underscores.
- Replace non-alphanumeric characters with underscores.
- Convert column names to lowercase.

In [5]:
# 2.1. Dealing with Duplicates
df.dropna(inplace=True)  

In [6]:
# 2.2. Standardize Column Names:
def format_column_titles(df):
    # Define a function to clean a single column name
    def clean_column(name):
        name = name.strip()  # Remove leading and trailing spaces
        # Replace non-alphanumeric characters with underscores
        name = re.sub(r'[^0-9a-zA-Z]+', '_', name)
        # Replace multiple underscores with a single underscore
        name = re.sub(r'_+', '_', name)
        name = name.lower()  # Convert to lowercase
        return name.strip('_')  # Remove leading and trailing underscores

    # Apply the clean_column function to all column names in the DataFrame
    df.columns = [clean_column(col) for col in df.columns]
    return df.columns
    
format_column_titles(df)
df.head()

,invoice_no,order_no,sales_rep_name,site,date,ship_to_country,market_segment,quantity,unit_cost,unit_price,total_profit_usd,total_cost_usd,total_value_usd
0,24002290,24100343,FBER,B,2024-06-28,US,TR,1464.0,10.2559,14.79,"$6,638","$15,015",21653
1,24002290,24100343,FBER,B,2024-06-28,US,TR,1.0,0.0000,0.00,$0,$0,0
2,24002292,24101537,FBER,B,2024-06-28,US,TR,100.0,3.6035,10.88,$728,$360,1088
3,24002292,24101537,FBER,B,2024-06-28,US,TR,1.0,0.0000,0.00,$0,$0,0
4,24002293,23103363,KTIE,B,2024-06-28,US,NT,21.0,14.2490,59.54,$951,$299,1250


### 3. Data Types Correction
- Ensure columns have the correct data types.
- Parse dates: Convert dates to datetime format.
1. Format Date to datetime
1. Format Quantity to int

In [7]:
# convert_date_to_datetime:
def convert_date_to_datetime(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    return df
convert_date_to_datetime(df, 'date')

,invoice_no,order_no,sales_rep_name,site,date,ship_to_country,market_segment,quantity,unit_cost,unit_price,total_profit_usd,total_cost_usd,total_value_usd
0,24002290,24100343,FBER,B,2024-06-28,US,TR,1464.0,10.2559,14.790,"$6,638","$15,015",21653
1,24002290,24100343,FBER,B,2024-06-28,US,TR,1.0,0.0000,0.000,$0,$0,0
2,24002292,24101537,FBER,B,2024-06-28,US,TR,100.0,3.6035,10.880,$728,$360,1088
3,24002292,24101537,FBER,B,2024-06-28,US,TR,1.0,0.0000,0.000,$0,$0,0
4,24002293,23103363,KTIE,B,2024-06-28,US,NT,21.0,14.2490,59.540,$951,$299,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,316985,24250233,MRHO,D,2024-06-03,US,PI,2.0,576.5250,2709.710,"$3,125",$848,3973
1115,316986,24250253,DHOU,D,2024-06-03,US,IC,1.0,420.5000,483.200,$45,$309,354
1116,316987,24250274,DHOU,D,2024-06-03,US,IC,1.0,420.5000,483.200,$45,$309,354
1117,316988,24250316,SGOY,D,2024-06-03,CA,TP,36.0,68.5469,289.780,"$8,618","$1,814",10432


In [8]:
# convert_quantity_to_integer
def convert_quantity_to_integer(df, quantity_col):
    df[quantity_col] = df[quantity_col].astype(int)
    return df

convert_quantity_to_integer(df, 'quantity')

,invoice_no,order_no,sales_rep_name,site,date,ship_to_country,market_segment,quantity,unit_cost,unit_price,total_profit_usd,total_cost_usd,total_value_usd
0,24002290,24100343,FBER,B,2024-06-28,US,TR,1464,10.2559,14.790,"$6,638","$15,015",21653
1,24002290,24100343,FBER,B,2024-06-28,US,TR,1,0.0000,0.000,$0,$0,0
2,24002292,24101537,FBER,B,2024-06-28,US,TR,100,3.6035,10.880,$728,$360,1088
3,24002292,24101537,FBER,B,2024-06-28,US,TR,1,0.0000,0.000,$0,$0,0
4,24002293,23103363,KTIE,B,2024-06-28,US,NT,21,14.2490,59.540,$951,$299,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,316985,24250233,MRHO,D,2024-06-03,US,PI,2,576.5250,2709.710,"$3,125",$848,3973
1115,316986,24250253,DHOU,D,2024-06-03,US,IC,1,420.5000,483.200,$45,$309,354
1116,316987,24250274,DHOU,D,2024-06-03,US,IC,1,420.5000,483.200,$45,$309,354
1117,316988,24250316,SGOY,D,2024-06-03,CA,TP,36,68.5469,289.780,"$8,618","$1,814",10432


In [9]:
unique_and_missing_values_dtype(df)

,non-Null_count,dtype,unique_values,%_unique,missing_values,%_missing
invoice_no,1117,int64,612,54.79%,0,0.0%
order_no,1117,int64,565,50.58%,0,0.0%
sales_rep_name,1117,object,15,1.34%,0,0.0%
site,1117,object,3,0.27%,0,0.0%
date,1117,datetime64[ns],20,1.79%,0,0.0%
ship_to_country,1117,object,9,0.81%,0,0.0%
market_segment,1117,object,22,1.97%,0,0.0%
quantity,1117,int64,177,15.85%,0,0.0%
unit_cost,1117,float64,637,57.03%,0,0.0%
unit_price,1117,float64,600,53.72%,0,0.0%


### 4. Data Correction
- 4.1. Identify data that need correction: values_count(), unique()
- 4.2. Correcting Typos, Handling Synonyms, Standardizing Formats 

In [10]:
# 4.1. Identify data that need correction: values_count(), unique()
# Visualize Value Counts for All Columns to identify outliers: 
def display_value_counts_all_columns(df):
    for col in df.columns:
        print(f"Value counts for column '{col}':")
        print(df[col].value_counts())
        print("\n" + "-"*50 + "\n")

display_value_counts_all_columns(df)
display_value_counts_all_columns

Value counts for column 'invoice_no':
invoice_no
24401222    20
24401155    20
317024      12
24401097    11
24401164    11
            ..
24401058     1
24401057     1
24401056     1
24401052     1
24001991     1
Name: count, Length: 612, dtype: int64

--------------------------------------------------

Value counts for column 'order_no':
order_no
24100343    20
24200186    20
23201826    20
24250424    12
24200326    11
            ..
24250233     1
24250253     1
24250274     1
24250316     1
24250381     1
Name: count, Length: 565, dtype: int64

--------------------------------------------------

Value counts for column 'sales_rep_name':
sales_rep_name
SBEA    211
MRHO    129
MWIS    119
DGAS    117
KTIE     75
FBER     74
AMIN     73
SGOY     70
BHOU     61
WMOR     46
GXAV     44
OGAL     44
THOU     32
JAKI     16
DHOU      6
Name: count, dtype: int64

--------------------------------------------------

Value counts for column 'site':
site
B    666
T    328
D    123
Name: count,

<function __main__.display_value_counts_all_columns(df)>

In [11]:
# 4.1. Identify data that need correction: values_count(), unique()
# Visualize Unique Value for All Columns to identify outliers: 
def display_value_counts_all_columns(df):
    for col in df.columns:
        print(f"Value counts for column '{col}':")
        print(df[col].unique())
        print("\n" + "-"*50 + "\n")

display_value_counts_all_columns(df)
display_value_counts_all_columns

Value counts for column 'invoice_no':
[24002290 24002292 24002293 24002294 24002295 24002296 24002297 24002298
 24002299 24002300 24002301 24002302 24002303 24002304 24002305 24002306
 24002307 24002308 24002309 24002310 24002311 24002312 24002313 24002314
 24002315 24002316 24002317 24002318 24002319 24002320 24002321 24002322
 24002323 24002324 24002325 24002326 24002327 24401219 24401220 24401221
 24401222 24401223 24401224 24401225 24401226 24401227   317072   317074
   317075   317076 24002275 24002276 24002277 24002278 24002279 24002280
 24002281 24002282 24002283 24002284 24002285 24002286 24002287 24002288
 24002289 24401207 24401208 24401209 24401210 24401211 24401212 24401213
 24401214 24401215 24401216 24401217 24401218   317066   317067   317068
   317069   317070 24002247 24002248 24002250 24002251 24002252 24002253
 24002254 24002255 24002256 24002257 24002258 24002259 24002260 24002261
 24002262 24002263 24002264 24002265 24002266 24002267 24002268 24002269
 24002270 240

<function __main__.display_value_counts_all_columns(df)>

In [12]:
# 4.2. Correcting Typos, Handling Synonyms, Standardizing Formats 
# Remove '$' Sign from Specified Columns
def remove_dollar_sign(df, columns):
    for col in columns:
        # Remove dollar sign, commas, and handle negative numbers in parentheses
        df[col] = df[col].replace({r'\$': '', r',': '', r'\(': '-', r'\)': ''}, regex=True)
        
        # Convert to float
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    return df

# Call the function with the necessary columns
df = remove_dollar_sign(df, ['total_profit_usd', 'total_cost_usd'])

In [13]:
# 4.2. Correcting Typos, Handling Synonyms, Standardizing Formats 
# Reduce Values to Two Decimal Places
def reduce_to_two_decimals(df, columns):
    for col in columns:
        df[col] = df[col].round(2)
    return df
reduce_to_two_decimals(df, ['unit_cost','unit_price'])

,invoice_no,order_no,sales_rep_name,site,date,ship_to_country,market_segment,quantity,unit_cost,unit_price,total_profit_usd,total_cost_usd,total_value_usd
0,24002290,24100343,FBER,B,2024-06-28,US,TR,1464,10.26,14.79,6638,15015,21653
1,24002290,24100343,FBER,B,2024-06-28,US,TR,1,0.00,0.00,0,0,0
2,24002292,24101537,FBER,B,2024-06-28,US,TR,100,3.60,10.88,728,360,1088
3,24002292,24101537,FBER,B,2024-06-28,US,TR,1,0.00,0.00,0,0,0
4,24002293,23103363,KTIE,B,2024-06-28,US,NT,21,14.25,59.54,951,299,1250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,316985,24250233,MRHO,D,2024-06-03,US,PI,2,576.52,2709.71,3125,848,3973
1115,316986,24250253,DHOU,D,2024-06-03,US,IC,1,420.50,483.20,45,309,354
1116,316987,24250274,DHOU,D,2024-06-03,US,IC,1,420.50,483.20,45,309,354
1117,316988,24250316,SGOY,D,2024-06-03,CA,TP,36,68.55,289.78,8618,1814,10432


In [14]:
unique_and_missing_values_dtype(df)

,non-Null_count,dtype,unique_values,%_unique,missing_values,%_missing
invoice_no,1117,int64,612,54.79%,0,0.0%
order_no,1117,int64,565,50.58%,0,0.0%
sales_rep_name,1117,object,15,1.34%,0,0.0%
site,1117,object,3,0.27%,0,0.0%
date,1117,datetime64[ns],20,1.79%,0,0.0%
ship_to_country,1117,object,9,0.81%,0,0.0%
market_segment,1117,object,22,1.97%,0,0.0%
quantity,1117,int64,177,15.85%,0,0.0%
unit_cost,1117,float64,536,47.99%,0,0.0%
unit_price,1117,float64,597,53.45%,0,0.0%


In [15]:
# 4.3. Export cleaned dataset
df_cleaned = df.to_csv('../GPT_OPEN_AI_project/data/cleaned/df_cleaned.csv')

### 5. OpenAI Library
- 5.1. Installation of the OpenAI Library
- 5.2. Reading API Configuration Keys

In [16]:
# 5.1. Installation of the OpenAI Library
#!pip install --upgrade openai

In [17]:
# 5.2. Reading API Configuration Keys
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv()

# Retrieve the API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

In [18]:
import dash
from dash import dcc, html
import openai
from dotenv import load_dotenv
import pandas as pd
import os

# Load environment variables from a .env file
load_dotenv()

# Ensure your API key is stored securely, ideally in an environment variable
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set your OPENAI_API_KEY environment variable.")

# Set the OpenAI API key
openai.api_key = api_key

# Example of how to make a request to OpenAI API
completion = openai.ChatCompletion.create(
    model="gpt-4o-mini",  # Make sure to use the appropriate model version
    messages=[
        {"role": "system", "content": "You are an analytical assistant, skilled in deriving insights from sales data."},
        {"role": "user", "content": "Provide a summary of the key trends in the sales data."}
    ]
)

# Print the response
print(completion.choices[0].message['content'])

To provide an effective summary of key trends in the sales data, I'd typically analyze various aspects, including overall sales growth, seasonal variations, product performance, customer demographics, and geographic distribution. Here’s how you might summarize these trends:

1. **Overall Sales Growth**:
   - Identify the percentage increase or decrease in sales year-over-year or quarter-over-quarter.
   - Note any significant milestones, such as record sales months or declines during specific periods.

2. **Seasonal Variations**:
   - Highlight peak sales periods (e.g., holidays, back-to-school).
   - Analyze off-peak seasons and any related strategies to increase sales during these times.

3. **Product Performance**:
   - Determine which products or categories are driving sales growth or experiencing declines.
   - Analyze the introduction of new products and their impact on sales.

4. **Customer Demographics**:
   - Examine changes in customer demographics or preferences.
   - Identi

In [19]:
completion

<OpenAIObject chat.completion id=chatcmpl-9vROhEdoxjGVPXTDqST4q2yEfKGfb at 0x11d514820> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "To provide an effective summary of key trends in the sales data, I'd typically analyze various aspects, including overall sales growth, seasonal variations, product performance, customer demographics, and geographic distribution. Here\u2019s how you might summarize these trends:\n\n1. **Overall Sales Growth**:\n   - Identify the percentage increase or decrease in sales year-over-year or quarter-over-quarter.\n   - Note any significant milestones, such as record sales months or declines during specific periods.\n\n2. **Seasonal Variations**:\n   - Highlight peak sales periods (e.g., holidays, back-to-school).\n   - Analyze off-peak seasons and any related strategies to increase sales during these times.\n\n3. **Product Performance**:\n   - Determine which produc

### 6. Build the Dashboard in DASH

In [20]:
# Load the cleaned sales data
df = pd.read_csv('../GPT_OPEN_AI_project/data/cleaned/df_cleaned.csv')

In [21]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [22]:
# Layout of the Dash app
app.layout = html.Div([
    html.H1("Interactive Sales Dashboard with AI Insights"),
    
    # Date range picker for filtering data
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['date'].min(),
        end_date=df['date'].max()
    ),
    
    # Dropdown to select different types of analysis
    dcc.Dropdown(
        id='analysis-type',
        options=[
            {'label': 'Total Sales Over Time', 'value': 'sales_over_time'},
            {'label': 'Sales by Sales Reps', 'value': 'top_sales_reps'}
        ],
        value='sales_over_time'
    ),
    
    # Graph to display the data
    dcc.Graph(id='sales-graph'),
    
    # Text box to input query for AI-generated insights
    dcc.Input(id='ai-query-input', type='text', placeholder='Ask about sales trends...', style={'width': '80%'}),
    html.Button('Get AI Insights', id='ai-query-btn', n_clicks=0),
    
    # Container for displaying AI-generated insights
    html.Div(id='ai-insights-output', style={'margin-top': '20px'}),
])


In [23]:
# Callback to update the graph based on selected analysis type and date range
@app.callback(
    Output('sales-graph', 'figure'),
    [Input('analysis-type', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graph(analysis_type, start_date, end_date):
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    if analysis_type == 'sales_over_time':
        fig = px.line(filtered_df, x='date', y='total_value_usd', title='Total Sales Over Time')
    
    elif analysis_type == 'top_sales_reps':
        if 'sales_rep_name' in df.columns:
            top_reps = filtered_df.groupby('sales_rep_name')['total_value_usd'].sum().reset_index().sort_values(by='total_value_usd', ascending=False)
            fig = px.bar(top_reps, x='sales_rep_name', y='total_value_usd', title='Sales by Sales Reps')
    
    return fig


# Callback to handle AI-generated insights based on the query input
@app.callback(
    Output('ai-insights-output', 'children'),
    [Input('ai-query-btn', 'n_clicks')],
    [State('ai-query-input', 'value'),
     State('analysis-type', 'value'),
     State('date-picker-range', 'start_date'),
     State('date-picker-range', 'end_date')]
)

def generate_ai_insights(n_clicks, query, analysis_type, start_date, end_date):
    if n_clicks > 0 and query:
        # Filter the data based on date range
        filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

        # Generate specific data summaries based on the selected analysis type
        if analysis_type == 'sales_over_time':
            total_sales = filtered_df['total_value_usd'].sum()
            summary = f"Total sales over the selected period is {total_sales:.2f} USD."
        
        elif analysis_type == 'top_sales_reps':
            top_reps = filtered_df.groupby('sales_rep_name')['total_value_usd'].sum().sort_values(ascending=False)
            summary = f"Top sales reps and their sales: {top_reps.head(5).to_dict()}."

        # Modify the query to include this summary
        extended_query = f"{query} Here is a summary of the data: {summary}"

        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",  # Replace with the appropriate model version
                messages=[
                    {"role": "system", "content": "You are an assistant specialized in analyzing sales data."},
                    {"role": "user", "content": extended_query}
                ]
            )
            return html.Div([
                html.H4("AI Insights"),
                html.P(response.choices[0]['message']['content'])
            ])
        except Exception as e:
            return html.Div([
                html.H4("Error"),
                html.P(f"An error occurred: {e}")
            ])
    return ""




### 7. Results. Run the app

In [24]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)